Make a Panel app to visualize the COVID19 data

Basic:
Allow users to choose countries. Months and data type to plot. If multiple countries are selected, color in different colors.
Bonus1:
Deploy on mybinder.org. 
share the URL link
Bonus2:
Add a Continent widget and update Countries options based on selected Continents
Add a table to display data for selected countries and months

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import panel as pn
import hvplot.pandas
from datetime import datetime

gene_pq = pd.read_csv("C:/Users/lch/Desktop/python课/world_wide_COVID_data_daily_data.csv").sort_values(by="geoId",ascending=False)  # .set_index("geoId")
countries_list = {}
for i in ['Africa', 'Asia', 'Europe', 'Oceania', 'America', 'Other']:
    countries_list[i] = list(set(gene_pq[gene_pq["continentExp"] == i]["countriesAndTerritories"].values))
continents_list = ['Africa', 'Asia', 'Europe', 'Oceania', 'America', 'Other']
gene_pq["dateRep"] = gene_pq["dateRep"].map(lambda x: "".join([x.split("/")[2], x.split("/")[1], x.split("/")[0]]))
gene_pq = gene_pq[gene_pq["year"] == 2020]
gene_pq = gene_pq.set_index("countriesAndTerritories")
pn.extension()

x = pn.widgets.RangeSlider(name='Months Range', start=1, end=12)#设置一系列按钮
y = pn.widgets.RadioButtonGroup(name='Button Group', options=['cases', 'deaths'])
continents = pn.widgets.Select(name="continents", options=continents_list)
countries = pn.widgets.MultiSelect(name="countries", options=countries_list[continents.value])


@pn.depends(continents, watch=True)
def update(continents):
    countries.options = countries_list[continents]
    return countries


@pn.depends(countries, x, y, watch=True)#传参给下面紧邻的函数，watch=True表示依赖的数据实时更新
def pic(countries, x, y):
    if countries == []:#一开始countries是一个空list，不加if判断会报错
        return None
    else:
        data_ori = gene_pq[gene_pq["month"] <= x[1]]
        date = data_ori[data_ori["month"] >= int(x[0])].loc[countries].sort_values(by="dateRep",ascending=True)#x[0]偶尔会显示出小数位使图表与实际不符，用int函数消除
        date["dateRep"] = date["dateRep"].map(lambda x: datetime.strptime(x, "%Y%m%d").date())#将dateRep转换为hvplot可识别的数字
        plot = date.hvplot(x="dateRep", y=y, by="countriesAndTerritories")
        return plot


@pn.depends(countries, x, watch=True)
def cssv(countries, x):
    if countries == []:
        return None
    else:
        data_ori = gene_pq[gene_pq["month"] <= x[1]]
        date = data_ori[data_ori["month"] >= int(x[0])].loc[countries].sort_values(by="dateRep",ascending=True)
        date["dateRep"] = date["dateRep"].map(lambda x: datetime.strptime(x, "%Y%m%d").date())
        return pn.pane.DataFrame(date, width=1200)

pn.Column(pn.Row(pn.WidgetBox(continents, update, x, y),pic),cssv).show()#传参后的函数可以不带后面括号直接用，默认输入depends()的数据

Launching server at http://localhost:55429
